In [1]:
import astropy.units as u
import astropy.constants as const
import numpy as np

import pandas as pd
import urllib
# the service URL
livechart = "https://nds.iaea.org/relnsd/v1/data?"

# There have been cases in which the service returns an HTTP Error 403: Forbidden
# use this workaround
import urllib.request
def lc_pd_dataframe(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

# Question 1

Typically, the efficiency of a gamma-ray detector is approximately $\varepsilon \approx 0.1$, and the minimum count rate is ${\rm CR}_{\min} \approx 0.5\,{\rm s^{-1}}$. What is the minimum detectable mass of $\rm ^{32}P$ in a sample ($\rm ^{32}P$, $T_{1/2} = 14.26 \, {\rm day}$)?

## Answer

The minimum count rate needed for a given radioactive source is $A_{\min}$

$$
A_{\min} = {\rm CR}_{\min} \varepsilon
$$

The relationship between radioactivity and the number of atoms is

$$
A = \lambda N
$$

Thus the minimal number of atoms needed is

$$
N_{\min} = \frac{{\rm CR}_{\min} \varepsilon}{\lambda}
$$

In [20]:
eps = 0.1
CR_min = 0.5*u.s**-1
T_half = 14.26*u.day
lam = np.log(2)/T_half
N_min = (CR_min*eps/lam).to(1)
N_min

<Quantity 88874.63114289>

So $N_{\min} = 8.89\times 10^5$. From which we can get the mass of the radioactive isotope:

$$
m = N\cdot m_{\rm ^{32}P}
$$

We can use data from Live Chart of Nuclides to get the mass of $\rm ^{32}P$ atoms(the unit of 'atomic_mass' columns is $\rm \mu u$):

In [17]:
P32_basic_property = lc_pd_dataframe(livechart+"fields=ground_states&nuclides=32P")
P32_basic_property[['z','n','symbol','jp','atomic_mass','massexcess']]

,z,n,symbol,jp,atomic_mass,massexcess
0,15,17,P,1+,3.197391e+07,-24304.876


In [25]:
m_32P = P32_basic_property['atomic_mass'][0]*u.uu
m_total = m_32P*N_min
m_total.to(u.g)

<Quantity 4.7187028e-18 g>

So the minimal detected mass of $\rm ^{32}P$ is $4.72\times 10^{-18}\,{\rm g}$.

# Question

Charcoal found in a deep layer of sediment in a cave is found to have an atomic $\rm ^{14}C/^{12}C$ ratio only 30% that of a charcoal sample from a higher level with a known age of 1850 years. What is the age of the deeper layer? （14C has a half-life of 5730 years, the ratio of $\rm ^{14}C$ to all carbon atoms in the environment in equilibrium was about $N_{\rm ^{14}C}/N_{\rm C} = 1.23 \times 10^{-12}$.）

## Answer

The ratio of $\rm ^{14}C/^{12}C$ of the charcoal sample with a age of 1850 years should be
$$
\frac{N_2(\rm^{14}C)}{N_2(\rm^{12}C)} = \frac{N_{\rm eq}(\rm^{14}C)}{N_{\rm eq}(\rm^{12}C)} \cdot {\rm e}^{-\lambda T_2} \\
T_2 = 1850\,{\rm yr}, \lambda = \frac{\ln 2}{T_{1/2}({\rm ^{14}C})}.
$$

And the ratio of $\rm ^{14}C/^{12}C$ of the unknown charcoal sample is
$$
\frac{N_1(\rm^{14}C)}{N_1(\rm^{12}C)} = \frac{N_{\rm eq}(\rm^{14}C)}{N_{\rm eq}(\rm^{12}C)} \cdot {\rm e}^{-\lambda T_1}
$$
where $T_1$ is its age.

We can divide these tow euqations to get the following equation:
$$
\frac{N_1(\rm^{14}C)}{N_1(\rm^{12}C)}\big / \frac{N_2(\rm^{14}C)}{N_2(\rm^{12}C)} = {\rm e}^{-\lambda(T_1 - T_2)}
$$

And we know that $\frac{N_1(\rm^{14}C)}{N_1(\rm^{12}C)}\big / \frac{N_2(\rm^{14}C)}{N_2(\rm^{12}C)} = 30\%$, so
$$
0.3 = {\rm e}^{-\lambda(T_1 - T_2)} = \frac 1{2^{\frac{T_1-T_2}{T_{1/2}}}} \\
T_1 = T_2 + T_{1/2}\log_2 \left(\frac{1}{0.3}\right)
$$

In [31]:
T2 = 1850*u.year
T_half = 5730*u.year
T1 = T2 + T_half*np.log2(1/0.3)
T1

<Quantity 11802.81285457 yr>

So the age of the unknwon sample is about 12 thousand years old.

# Question 3

What type of a radioactive source should be used, and how should it be used in thickness gauging processes if it is to be used for 

(a) gauging paper thickness, 

(b) the control of sheet metal thickness between 0.1 and 1 cm, 

and (c) controlling the coating thickness of adhesive on a cloth substrate?

## Answer

Gauging paper thickness is well suited to beta particle sources. The greater mass thickness of sheet metal requires x ray or gamma ray sources. Controlling coating thickness is a special challenge because the coating is often similar to the substrate in density and atomic mass. In that case, flourescence thickness gauging is called for.